In [ ]:
# 1. New detection variables
lh = []
rh = []
sentence = []
threshold = 0.6
now_sign=[]
font=cv2.FONT_HERSHEY_SIMPLEX
frame_num_array=[]
color=(0,255,0)
size = 1
thickness=2
while True:
    frame, hands, bag = tracker.next_frame()
    if frame is None: break
    # Draw hands
    frame_num = tracker.fps.nb_frames()
    frame_num_array.append(frame_num)
    frame_count = frame_num - frame_num_array[0]
    # predictions logic
    if frame_count > 0  : 
        if Decimal(frame_count/30) != int(frame_count/30) :
            cv2.putText(frame,'Detecting',(15,30)
                                     , cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
            hand = renderer.draw(frame, hands, bag)
            key = renderer.waitKey(delay=1)
            lh_data,rh_data = extractData(hands)
            lh.append(lh_data)
            rh.append(rh_data)
        if Decimal(frame_count/30) == int(frame_count/30):
            lh_used=lh[-30:]
            rh_used=rh[-30:]
            lrh = np.concatenate([lh_used,rh_used])
            lrh_used = lrh.reshape(lrh.shape[0],-1)
            if len(lrh) == 60:
                res = sign_model.predict(np.expand_dims(lrh_used, axis=0))[0]
                predicted_sign = actions[np.argmax(res)]
                confidence = res[np.argmax(res)]
                if (confidence > threshold) & (predicted_sign not in now_sign)  : 
                    print(predicted_sign)
                    print(confidence)
                    cv2.putText(frame,'{}'.format(predicted_sign),(15,30)
                                     , cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    now_sign=[actions[np.argmax(res)]]
            cv2.waitKey(50)
            
        lh = []
        rh = []
        break
renderer.exit()